In [1]:
import requests
import json
from datetime import datetime

# -----------------------------
# Configuration
# -----------------------------
API_BASE_URL = "https://api.marketdata.app/v1"
# You may need an API key if required by the endpoint
API_KEY = "YWZ4ZWJVLWNrYlQyZGpYUnpLYUtXZjlhY3ptei1zcjNsbk5fMDg5NXdwZz0"  # Optional, if you have a token

# -----------------------------
# Helper Functions
# -----------------------------

def get_stock_quote(symbol):
    """
    Get real-time stock quote for the given symbol.
    """
    url = f"{API_BASE_URL}/stocks/quotes/{symbol}/"
    headers = {}
    if API_KEY:
        headers["Authorization"] = f"Bearer {API_KEY}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error fetching quote: {response.status_code} {response.text}")
        return None

def get_historical_candles(symbol, resolution="D", from_date=None, to_date=None, countback=None):
    """
    Get historical stock candles.
    resolution: '1' = 1 min, 'D' = daily, 'W' = weekly, 'M' = monthly
    from_date / to_date: YYYY-MM-DD format
    countback: number of candles from 'to_date' backwards
    """
    url = f"{API_BASE_URL}/stocks/candles/{resolution}/{symbol}/"
    params = {}
    if from_date:
        params["from"] = from_date
    if to_date:
        params["to"] = to_date
    if countback:
        params["countback"] = countback
    params["format"] = "json"

    headers = {}
    if API_KEY:
        headers["Authorization"] = f"Bearer {API_KEY}"

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching historical candles: {response.status_code} {response.text}")
        return None

# -----------------------------
# Example Usage
# -----------------------------

if __name__ == "__main__":
    # Replace with any stock symbol you like
    stock_symbol = "AAPL"

    # Get real-time quote
    quote = get_stock_quote(stock_symbol)
    if quote:
        print(f"\nReal-Time Quote for {stock_symbol}:")
        print(json.dumps(quote, indent=4))

    # Get historical daily candles for 2023
    candles = get_historical_candles(stock_symbol, resolution="D", from_date="2023-01-01", to_date="2023-12-31")
    if candles:
        print(f"\nHistorical Daily Candles for {stock_symbol}:")
        print(json.dumps(candles, indent=4))


Error fetching quote: 203 {"s":"ok","symbol":["AAPL"],"ask":[271.49],"askSize":[100],"bid":[271.35],"bidSize":[1900],"mid":[271.42],"last":[271.49],"change":[5.24],"changepct":[0.0197],"volume":[59200384],"updated":[1763773199]}

Historical Daily Candles for AAPL:
{
    "s": "ok",
    "t": [
        1672722000,
        1672808400,
        1672894800,
        1672981200,
        1673240400,
        1673326800,
        1673413200,
        1673499600,
        1673586000,
        1673931600,
        1674018000,
        1674104400,
        1674190800,
        1674450000,
        1674536400,
        1674622800,
        1674709200,
        1674795600,
        1675054800,
        1675141200,
        1675227600,
        1675314000,
        1675400400,
        1675659600,
        1675746000,
        1675832400,
        1675918800,
        1676005200,
        1676264400,
        1676350800,
        1676437200,
        1676523600,
        1676610000,
        1676955600,
        1677042000,
       

In [42]:
import requests
import numpy as np
import pandas as pd
import json

# -----------------------------
# Configuration
# -----------------------------
API_BASE_URL = "https://api.marketdata.app/v1"
API_KEY = "YWZ4ZWJVLWNrYlQyZGpYUnpLYUtXZjlhY3ptei1zcjNsbk5fMDg5NXdwZz0"  # Optional, if you have a token
STOCK_SYMBOL = "MSFT"

# -----------------------------
# Fetch historical daily candles
# -----------------------------
def get_historical_candles(symbol, resolution="D", from_date=None, to_date=None):
    url = f"{API_BASE_URL}/stocks/candles/{resolution}/{symbol}/"
    params = {
        "format": "json"
    }
    if from_date:
        params["from"] = from_date
    if to_date:
        params["to"] = to_date

    headers = {}
    if API_KEY:
        headers["Authorization"] = f"Bearer {API_KEY}"

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching historical candles: {response.status_code} {response.text}")
        return None

# -----------------------------
# Process data into NumPy arrays and indicators
# -----------------------------
dat = get_historical_candles(STOCK_SYMBOL, from_date="2025-08-25", to_date="2025-11-21")


In [43]:
# your MarketData.app JSON
data_json = dat  # <-- replace with your JSON

# Build DataFrame directly from the lists
df = pd.DataFrame({
    "open": data_json["o"],
    "high": data_json["h"],
    "low": data_json["l"],
    "close": data_json["c"],
    "volume": data_json["v"]
})

# Ensure ascending order by timestamp
df['t'] = data_json["t"]
df = df.sort_values('t').reset_index(drop=True)

# -----------------------------
# Calculate indicators
# -----------------------------
df['avg_volume'] = df['volume'].rolling(window=20).mean()
df['adr'] = df['high'] - df['low']
df['ma10'] = df['close'].rolling(window=10).mean()
df['ma20'] = df['close'].rolling(window=20).mean()
df['ma50'] = df['close'].rolling(window=50).mean()

# VWAP
tp = (df['high'] + df['low'] + df['close']) / 3
df['vwap'] = (tp * df['volume']).cumsum() / df['volume'].cumsum()

# RSI (14-day)
delta = df['close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(window=14).mean()
avg_loss = loss.rolling(window=14).mean()
rs = avg_gain / avg_loss
df['rsi'] = 100 - (100 / (1 + rs))

# Replace NaNs with zeros
df.fillna(0, inplace=True)

# -----------------------------
# Convert to NumPy arrays
# -----------------------------
opens = df['open'].to_numpy()
highs = df['high'].to_numpy()
lows = df['low'].to_numpy()
closes = df['close'].to_numpy()
volumes = df['volume'].to_numpy()
adr = df['adr'].to_numpy()
vwap = df['vwap'].to_numpy()
avg_volume = df['avg_volume'].to_numpy()
ma10 = df['ma10'].to_numpy()
ma20 = df['ma20'].to_numpy()
ma50 = df['ma50'].to_numpy()
rsi = df['rsi'].to_numpy()

# -----------------------------
# Extras array (all indicators + OHLCV)
# -----------------------------
extras_np = np.column_stack([
    opens, highs, lows, closes, volumes,
    avg_volume, adr, vwap, ma10, ma20, ma50, rsi
])

print("Extras array shape:", extras_np.shape)  # (num_days, 12)
print(extras_np)  # first 5 rows

Extras array shape: (64, 12)
[[5.06630000e+02 5.08190000e+02 5.04120000e+02 5.04260000e+02
  2.16385790e+07 0.00000000e+00 4.07000000e+00 5.05523333e+02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.04355000e+02 5.04977800e+02 4.98510000e+02 5.02040000e+02
  3.08357090e+07 0.00000000e+00 6.46780000e+00 5.03360407e+02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.02000000e+02 5.07290000e+02 4.99900000e+02 5.06740000e+02
  1.72778930e+07 0.00000000e+00 7.39000000e+00 5.03678193e+02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.07090000e+02 5.11090000e+02 5.05500000e+02 5.09640000e+02
  1.80155930e+07 0.00000000e+00 5.59000000e+00 5.04717886e+02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.08660000e+02 5.09600000e+02 5.04491500e+02 5.06690000e+02
  2.09615690e+07 0.00000000e+00 5.10850000e+00 5.05143806e+02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [5.00465000e+02 5.06000000e+02 4.96